## Simple Vacuum Adaptor

My shop has vacuum hose that doesn't exactly fit the dust ports on many of my machines. I desiged and printed an adaptor to create a firm connection:

In [1]:
from petrify import u
from petrify.solid import Point, Vector, Box, Cylinder, Node, Union
from petrify.edge import Chamfer

This is a great example to demonstrate how petrify's python backing lets us create re-usable components:

In [2]:
fidelity = 50

class Tube(Node):
    def __init__(self, inner_diameter, outer_diameter, height):
        self.outer = Cylinder(
            Vector(0, 0, 0),
            Vector(0, 0, height),
            outer_diameter / 2,
            segments = fidelity
        )

        self.inner = Cylinder(
            Vector(0, 0, 0),
            Vector(0, 0, height),
            inner_diameter / 2,
            segments = fidelity
        )

        super().__init__((self.outer - self.inner).polygons)

Let's define the parameters we need:

In [3]:
units = u.inches

port = 2.65
outer = 2.28
inner = 2.22
wall = 0.125

And now, let's start making some geometry. I initially just fused together two tubes for the input and output sides:

In [4]:
smaller = Tube(inner - wall * 2, inner, 1.5)
bigger = Tube(inner - wall * 2, outer, 1.5) + Vector(0, 0, 1.5 - wall)

(smaller + bigger).visualize()

Preview(child=BufferGeometry(attributes={'position': <BufferAttribute shape=(3780, 3), dtype=float32>, 'normal…

This worked _okay_ in testing. It fit everything well, but didn't stay put. I decided to create a holster port that would snug the adaptor up to the dust port and keep it in place. I also threw in a chamfer for good measure:

In [5]:
holster_top = Tube(inner - wall, port + wall * 2, 0.75)
outside = holster_top.outer
chamfer = Chamfer(outside, outside.top.segments(), port - outer)
holster_top = holster_top - chamfer
holster = Tube(port, port + wall * 2, 1.25) + holster_top.translate(Vector(0, 0, 1.0))

adaptor = smaller + holster + bigger
adaptor.visualize()

Preview(child=BufferGeometry(attributes={'position': <BufferAttribute shape=(16050, 3), dtype=float32>, 'norma…

Looks good, let's convert it to STL for slicing and printing:

In [6]:
from petrify.formats import STL

STL('../output/vacuum-adaptor.stl', u.mm).write(adaptor.as_unit(units))